In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import mean_squared_error
import pymysql, calendar, time, json
import datetime
import pandas as pd

class DBread:
    sql_news='SELECT * FROM news_team_4.tb_news;'
    sql_comment='SELECT * FROM news_team_4.tb_comment;'
    sql_user='SELECT * FROM news_team_4.tb_user;'
    sql_nocomment='select * from news_team_4.tb_news tnt where id in (select NewsID  from news_team_4.tb_comment tc group by NewsID );'
    How_many=400000
    def __init__(self):
        
        self.conn = pymysql.connect(host='localhost', user='root',
            password='', db='', charset='utf8',cursorclass=pymysql.cursors.DictCursor)

    def data_news(self):
        
        with self.conn.cursor() as curs:
            curs.execute(self.sql_nocomment)
            # rs=curs.fetchmany(self.How_many)
            rs=curs.fetchall()
        news=pd.DataFrame(rs)
        return news
    def data_comment(self):

        with self.conn.cursor() as curs:
            curs.execute(self.sql_comment)
            # rs=curs.fetchmany(self.How_many)
            rs=curs.fetchall()

        comment=pd.DataFrame(rs)
        return comment
    def data_user(self):
        with self.conn.cursor() as curs:
            curs.execute(self.sql_user)
            # rs=curs.fetchmany(self.How_many)
            rs=curs.fetchall()

        user=pd.DataFrame(rs)
        return user

    def __del__(self):
       self.conn.close()
        
if __name__ == '__main__':
    news = DBread().data_news()
    comment=DBread().data_comment()
    # user=DBread().data_user()
    
    

Exception ignored in: <function DBread.__del__ at 0x7fe0d31ee950>
Traceback (most recent call last):
  File "/var/folders/px/2gqqh24s6yb9zp3q9m6th5f80000gn/T/ipykernel_1312/2936170907.py", line 43, in __del__
AttributeError: 'DBread' object has no attribute 'conn'
Exception ignored in: <function DBread.__del__ at 0x7fe0d328b830>
Traceback (most recent call last):
  File "/var/folders/px/2gqqh24s6yb9zp3q9m6th5f80000gn/T/ipykernel_1312/430468594.py", line 43, in __del__
AttributeError: 'DBread' object has no attribute 'conn'


In [10]:

def get_rmse(R, P, Q, non_zeros):
    error = 0
    # 두개의 분해된 행렬 P와 Q.T의 내적으로 예측 R 행렬 생성
    full_pred_matrix = np.dot(P, Q.T)
    
    # 실제 R 행렬에서 널이 아닌 값의 위치 인덱스 추출하여 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
      
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    
    return rmse


def matrix_factorization(R, K, steps=200, learning_rate=0.01, r_lambda = 0.01):
    num_users, num_items = R.shape
    # P와 Q 매트릭스의 크기를 지정하고 정규분포를 가진 랜덤한 값으로 입력합니다. 
    np.random.seed(1)
    P = np.random.normal(scale=1./K, size=(num_users, K))
    Q = np.random.normal(scale=1./K, size=(num_items, K))

    break_count = 0
       
    # R > 0 인 행 위치, 열 위치, 값을 non_zeros 리스트 객체에 저장. 
    non_zeros = [ (i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] > 0 ]
   
    # SGD기법으로 P와 Q 매트릭스를 계속 업데이트. 
    for step in range(steps):
        for i, j, r in non_zeros:
            # 실제 값과 예측 값의 차이인 오류 값 구함
            eij = r - np.dot(P[i, :], Q[j, :].T)
            # Regularization을 반영한 SGD 업데이트 공식 적용
            P[i,:] = P[i,:] + learning_rate*(eij * Q[j, :] - r_lambda*P[i,:])
            Q[j,:] = Q[j,:] + learning_rate*(eij * P[i, :] - r_lambda*Q[j,:])
       
        rmse = get_rmse(R, P, Q, non_zeros)
        if (step % 10) == 0 :
            print("### iteration step : ", step," rmse : ", rmse)
            
    return P, Q

def matrix(data):
    P, Q = matrix_factorization(data.values, K=50, steps=300, learning_rate=0.01, r_lambda = 0.01)
    pred_matrix = np.dot(P, Q.T)

    return pred_matrix

def recommend_list(matrix,userID,number_user):
    compare=matrix.loc[userID, :].sort_values(ascending=False)[1:number_user]
    compare_lists=list(compare.index)
    Recommend_list=[]
    for compare_list in compare_lists:
        New_one=list(coll_user_news.loc[coll_user_news.UserID==compare_list].NewsID.astype('str'))
        Already_seen=list(coll_user_news.loc[coll_user_news.UserID==userID].NewsID.astype('str'))

        for News in range(len(New_one)):
            
            if New_one[News]  not in Already_seen:
                Recommend_list.append(int(New_one[News]))

    return Recommend_list
    
def get_unseen_news(ratings_matrix, userId):
    user_rating = ratings_matrix.loc[userId,:]
    
    already_seen = user_rating[ user_rating == 1].index.tolist()
    
    news_list = ratings_matrix.columns.tolist()
    
    
    unseen_list = [ news for news in news_list if news not in already_seen]
    
    return unseen_list

news=news.loc[(news.DomainID==2)]
coll_news=news[['ID','Content']].rename(columns={'ID':'NewsID'}) # 여기서 ID 는 news id
coll_comment=comment[['UserID','NewsID']]
coll_user_news=pd.merge(coll_comment,coll_news,on='NewsID')
coll_user_news.dropna(inplace=True)
coll_user_news['rating']=1
coll_user_news
coll_user_news.Content.replace('',np.nan,inplace=True)
coll_user_news.dropna(axis=0,inplace=True)






In [18]:
data = coll_user_news.pivot_table('rating', index = 'UserID', columns = 'NewsID')# data = data.transpose()




In [19]:
news_sim_df=matrix(data)
ratings_pred_matrix = pd.DataFrame(data=news_sim_df, index= data.index,
                                columns = data.columns)

In [97]:
Data_save=pd.DataFrame(index=ratings_pred_matrix.index,columns=['recommend'])

for rating in range(len(ratings_pred_matrix.index)):
    
    unseen_list = get_unseen_news(data,ratings_pred_matrix.index[rating])
    recomm_news = list(ratings_pred_matrix.loc[ratings_pred_matrix.index[rating], unseen_list].sort_values(ascending=False)[:10].index)
    Data_save.loc[ratings_pred_matrix.index[rating],'recommend']=recomm_news

Data_save['UserID']=Data_save.index
Data_save.drop(columns='UserID',inplace=True)
total_Data=pd.merge(comment,Data_save,on='UserID')


Exception ignored in: <function DBread.__del__ at 0x7fe0d328bf80>
Traceback (most recent call last):
  File "/var/folders/px/2gqqh24s6yb9zp3q9m6th5f80000gn/T/ipykernel_1312/430468594.py", line 43, in __del__
AttributeError: 'DBread' object has no attribute 'conn'


In [123]:
total_Data.to_csv('total2.csv')

In [149]:
# print(coll_user_news.loc[(coll_user_news.UserID==21597)].Content)
# print(coll_user_news.loc[(coll_user_news.NewsID==591068)].Content)#.iloc[0]


Series([], Name: Content, dtype: object)
21596    박지영 기자 “카메라 얼마나 더 좋아지려고갤럭시S23 울트라 ‘뒤태’ 보니”삼성전자...
21597    박지영 기자 “카메라 얼마나 더 좋아지려고갤럭시S23 울트라 ‘뒤태’ 보니”삼성전자...
21598    박지영 기자 “카메라 얼마나 더 좋아지려고갤럭시S23 울트라 ‘뒤태’ 보니”삼성전자...
21599    박지영 기자 “카메라 얼마나 더 좋아지려고갤럭시S23 울트라 ‘뒤태’ 보니”삼성전자...
21600    박지영 기자 “카메라 얼마나 더 좋아지려고갤럭시S23 울트라 ‘뒤태’ 보니”삼성전자...
                               ...                        
21717    박지영 기자 “카메라 얼마나 더 좋아지려고갤럭시S23 울트라 ‘뒤태’ 보니”삼성전자...
21718    박지영 기자 “카메라 얼마나 더 좋아지려고갤럭시S23 울트라 ‘뒤태’ 보니”삼성전자...
21719    박지영 기자 “카메라 얼마나 더 좋아지려고갤럭시S23 울트라 ‘뒤태’ 보니”삼성전자...
21720    박지영 기자 “카메라 얼마나 더 좋아지려고갤럭시S23 울트라 ‘뒤태’ 보니”삼성전자...
21721    박지영 기자 “카메라 얼마나 더 좋아지려고갤럭시S23 울트라 ‘뒤태’ 보니”삼성전자...
Name: Content, Length: 126, dtype: object
